# 🛰️ Time to Focus Our SAR Image! 🎯  

We've got our **SAR data**—now it's time to turn it into a **sharp, high-resolution radar image**! 🖼️✨  

The key? **Focusing the echoes** we've collected along our flight path into a **sharp beam**. 🎯 As we just learned, we need to carefully account for the **changing distances** between the radar along the synthetic aperture and every point in the scene. 📡📍  

This is where our hero comes in: **Backprojection Beamforming**! 🔍⚡  

By properly aligning and combining the signals from all our virtual antenna elements, we can **reconstruct a detailed image of the ground**.  

---

### 🔲 Setting Up Our Image Grid 📡🗺️  

What are we trying to create as an image?  

We want a **2D map** that shows us **exactly where the microwave energy we received actually came from**. 🌌✨  

But before we can process the data, we need to decide **where to focus our wave energy**. 

Naturally, we’re interested in the area we **illuminated with our radar antenna** during the data collection. 🔦  

---

### 1️⃣ **Define the Image Area** 🖼️  
   - First, we select a Cartesian **(x, y)** region on the ground where we want to reconstruct our image.   
   - This area should be within the region illuminated by the radar beam. 🔦📡  
   - Think of it as setting the boundaries for our SAR “painting”. 🎨  
   
#### 📏 **How the Axes Are Defined:**  
   - **y-axis (Ground Range):**  
     - This is the **line-of-sight vector** from the **midpoint of the synthetic aperture projected onto the ground plane**.  
     - It represents the **distance from the radar** to points on the ground along the line of sight.  

   - **x-axis (Cross Range):**  
     - This axis runs **along the flight path**, **perpendicular to the ground range (y-axis)** and the **ground surface normal**.  
     - It represents the **location along the synthetic aperture**, providing the second dimension of our image.  

   - Together, these axes form a **2D coordinate system** where each pixel represents a small area on the ground, ready to be focused into a SAR image. 🖌️✨  

---

### 2️⃣ **Divide It Into Pixels** 🔳  
   - Next, we break this area into a **grid of tiny resolution cells (pixels)**.  
   - 📏 **Ground Range (y) Spacing:** Determined by the **range resolution** and the incidence angle. (We'll dive more into the geometry in the next lesson)
   - 📏 **Cross-Range (x) Spacing:** Determined by the **cross-range resolution**, which depends on the **synthetic aperture length** and the distance to the target.  

---

### 🌌 **Illustrating Our Coordinate System**  
Below is a visualization of our **coordinate system**, showing how the **ground range (y-axis)** and **cross-range (x-axis)** are defined in relation to the radar’s flight path and the ground plane. 📐🛰️✨  

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

# For illustration, let's define a small SAR geometry example
number_of_pulses = 10
x_radar = np.linspace(-10, 10, number_of_pulses)
y_radar = 100
z_radar = 100
xyz_radar_vs_pulse = np.column_stack((x_radar, np.full(number_of_pulses, y_radar), np.full(number_of_pulses, z_radar)))

# Define the 30x30 ground rectangle centered at the origin
rect_x = [ -15,  15,  15, -15, -15 ]
rect_y = [ -15, -15,  15,  15, -15 ]
rect_z = [   0,   0,   0,   0,   0 ]

# 3D Plot
fig = plt.figure(figsize=(8, 6))
ax = fig.add_subplot(111, projection='3d')

# Radar trajectory
ax.plot(
    xyz_radar_vs_pulse[:, 0],
    xyz_radar_vs_pulse[:, 1],
    xyz_radar_vs_pulse[:, 2],
    'o-', label='Radar Trajectory (Synthetic Aperture)'
)

# Extract the first and last radar positions
start_point = xyz_radar_vs_pulse[0, :]
end_point = xyz_radar_vs_pulse[-1, :]
# Calculate the direction vector for the quiver (end - start)
vector = end_point - start_point
# Add a quiver arrow from start to end
ax.quiver(
    start_point[0], start_point[1], start_point[2],  # Starting point
    vector[0], vector[1], vector[2],                  # Direction vector
    color='blue', arrow_length_ratio=0.25, linewidth=2,
)

# Plot the 30x30m ground rectangle at z=0
ax.plot(rect_x, rect_y, rect_z, 'k-', linewidth=2, label='Imaged Area (Ground)')

# Axis on the ground plane
center_pt = np.array([0, 0, 0])
arrow_x_end = np.array([25, 0, 0])   # cross-range
arrow_y_end = np.array([0, -25, 0])  # ground-range (opposite direction)

ax.quiver(
    center_pt[0], center_pt[1], center_pt[2],
    arrow_x_end[0], arrow_x_end[1], arrow_x_end[2],
    color='red', arrow_length_ratio=0.25, linewidth=2,
)
ax.quiver(
    center_pt[0], center_pt[1], center_pt[2],
    arrow_y_end[0], arrow_y_end[1], arrow_y_end[2],
    color='blue', arrow_length_ratio=0.25, linewidth=2
)

# Label the axes near the arrow tip
ax.text(
    arrow_x_end[0], arrow_x_end[1], arrow_x_end[2] + 10,
    "X (Cross-Range)", color='red', fontsize=10
)
ax.text(
    arrow_y_end[0], arrow_y_end[1]-40, arrow_y_end[2] + 10,
    "Y (Ground Range)", color='blue', fontsize=10
)


# Line-of-Sight (LOS) Vector: we'll connect the midpoint of the radar aperture to the scene center (0,0,0).
mid_index = len(xyz_radar_vs_pulse) // 2
radar_mid = xyz_radar_vs_pulse[mid_index]  # e.g. x,y,z
scene_center = np.array([0, 0, 0])

los_vector = scene_center - radar_mid
ax.quiver(
    radar_mid[0], radar_mid[1], radar_mid[2],
    los_vector[0], los_vector[1], los_vector[2],
    color='green', arrow_length_ratio=0.05,
    linewidth=2, label="Line of Sight (Slant Range)"
)

# Axis limits
ax.set_xlim3d([-50, 50])
ax.set_ylim3d([-50, 150])
ax.set_zlim3d([0, 150])

ax.set_xlabel("X (m)")
ax.set_ylabel("Y (m)")
ax.set_zlabel("Z (m)")
ax.set_title("Radar Trajectory & SAR Image Axes")

ax.legend()
plt.tight_layout()
plt.show()

The task now? To **process all the received echoes** and figure out how much wave energy was reflected by each pixel location. 🔍📸  

### 📍 How to Focus the Energy for One Pixel Bucket 🪣✨  

Imagine that **each pixel in the SAR image is a bucket** waiting to be filled with microwave energy. 🪣〰️

Our job? To **collect all the energy** that originated from that pixel’s location and **deposit it into the correct bucket**. Once we’ve filled all the buckets, we’ll have our SAR image! 🎉🖼️  

But here’s the challenge: How do we make sure we’re putting the **right energy in the right bucket**? 🤔  

Let's pick **one pixel position** and start to fill the bucket! 🪣🌊

The idea is simple:  

We need to **find, align, and coherently combine** all the signals that were **reflected from this specific pixel location**. 📍💥  
If we do this correctly, we’ll fill the bucket with the right amount of energy and make our pixel shine! ✨  

Here's how we do it, step-by-step:  

### 1️⃣ **Calculate the Range (Time Delay)** 📏⏳  

Every time the radar **transmits a pulse**, it travels all the way to the pixel location and some energy is reflected back. 🔦🪞  

The energy reflected by the pixel location appears at a certain **time delay** in our data, determined by the **distance between the radar and the pixel location** at the given measurement position.

And thanks to the amazing work of the **OD and ADCS teams** 🦸‍♂️🦸‍♀️, we know the **position of the radar** for each transmitted pulse (i.e. the locations of our virtual antenna elements)! 🚀  

We can use this information to **figure out the right time delays (and ranges)** for the pixel echoes!

---

#### 📍 How We Calculate the Range  
For a pixel located at $(x_p, y_p, z_p)$, the range from the radar position $(x_r(t), y_r(t), z_r(t))$ is:  

$$
R_p(t) = \sqrt{(x_p - x_r(t))^2 + (y_p - y_r(t))^2 + (z_p - z_r(t))^2}
$$  

What does this mean? 🤔  
- This formula gives us the **distance between the pixel and each virtual antenna element** (pulse).
- We can use the pixel distance to calculate the **path length difference of the echo** at each virtual element position. 📏
- The different **virtual antenna elements** correspond to different time $t$ values along our **orbital trajectory**.  
- The crucial point: The range (or time delay) **changes** as the radar moves along its flight path, from one pulse to the next. 🔄✈️  

This **variation in range** is the key to forming our SAR image! 📸✨  

---

### 2️⃣ **Pick the Right Samples to Sum** 🍌  
Once we know the **range to the pixel** for each pulse, we can figure out **which range (time-delay) samples in our recorded data correspond to echoes from that pixel**.  
- From each virtual antenna element (pulse), we **pick the sample corresponding to the calculated range**, tracing that tasty banana curve! 🍌

---

### 3️⃣ **Adjust the Phases for Coherent Beamforming** 🔄🌀  

Alright, we've picked the **right samples** from each virtual element signal. Now, we need to make sure the signals will **add up coherently** before combining them to form our focused beam. 💪✨  

Here's the problem:  
- The echo signals from our pixel have traveled **different distances** for each pulse. 🚀  
- If we just added them up as they are, they'd sum in random phases and our beam would not focus properly. ❌  

✅ **What we need to do:** Apply a **phase correction** to each signal based on the calculated range to the pixel. This ensures that all the signals will add up constructively, producing a strong focused signal. 🌟  

The phase correction is calculated as:  
$$
\text{Corrected Signal} = \text{Received Signal} \times e^{-j 2\pi \frac{2 R_p(t)}{\lambda}}
$$  

What are we doing here? 🤓  
- We're calculating the **exact path length**  $2\cdot R_p(t)$ for each signal received by the different virtual elements.  
- This is very similar to what we already did in **Lesson 3**! The difference is, we're **not making any plane wave or far-field assumptions**.  
- Instead, we calculate the **precise path length** $2\cdot R_p(t)$ for the pixel location where we are focusing the beam. 📏📡✨  

Once the signals are **phase-corrected**, we're ready to **sum them up and fill our bucket** to form our focused pixel value. 🪣✨  

---

### 4️⃣ **Sum Them Up** ➕💥  

Now that we’ve **picked the echoes from correct time delays and adjusted the phases**, it’s time to **combine them and see the magic of beamforming happen!** ✨  

Here's what we do:  
- We **sum up all the corrected signals** from every virtual element (pulse). ↗️➕↗️➕↗️➕↗️  
- This process ensures **coherent summation** for echoes coming from our pixel location.  
- Meanwhile, signals from other areas will **add incoherently and mostly cancel out**. ↖️➕↙️➕↗️➕↘️ 

What do we get in the end?  
- A **single phasor value** representing the microwave energy received from this pixel location on the ground. 🎯  
- It contains both **amplitude and phase information**, forming a sample in our **single-look-complex (SLC) SAR image**. 📊✨  

We've now filled the bucket 🪣🌊 with wave energy for **one pixel**!  

---

### 5️⃣ **Repeat for All Pixels** 🔁🗺️  

Now that we understand how to **focus our beam on a single pixel location**, it’s time to **repeat the magic** for every pixel in our desired imaging area! 🔄✨  

Here's the process:  
1. **Pick a pixel position**. 📍  
2. **Calculate the range** to that pixel from all our virtual antenna elements. 📏
3. **Pick the correct echo samples** from each pulse based on the calculated range. 🧺
4. **Apply phase corrections** to each of the virtual element phasors that you picked. 🔄  
5. **Add them up coherently** to get a focused value for that pixel. ➕  

And then... we do it again! And again! And again! 🔁  

The end result?  
- A beautiful **2D map**, where each pixel tells us **how much energy was reflected from that specific location**. 🌟🖼️  
- And voilà! We've created a **high-resolution SAR image**. 🎉🚀  

---

Time to bring all the theory to life! 🌟 Let’s **put all the pieces together** and see how this works in practice! 🔧💪  

First, we’ll **simulate some range-compressed echo data** collected with our hypothetical SAR system. 📡💥  

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Radar & Simulation Parameters
c = 299792458          # Speed of light, m/s
range_resolution = 1.0 # m
wavelength = 0.3       # m
B = c / (2 * range_resolution)  # Chirp bandwidth, Hz
sampling_rate = 8 * B         # Hz
pulse_duration = 10e-6   # s
pulse_width = pulse_duration * c  # m

# You can adjust this as needed to see more or fewer pulses:
number_of_pulses = 500  

x_start, x_end = -20.0, 20.0  # Aperture beginning and end (m)
y_radar = 100.0               # Radar y coordinate (m)
z_radar = 100.0               # Radar z coordinate (m)

# Define multiple targets here
# Each target has (x,y) and an optional amplitude (reflection strength).
targets = [
    {"x": 0.0,   "y": 0.0,  "amplitude": 1.0},
    {"x": 5.0,   "y": 10.0, "amplitude": 0.5},
    {"x": -10.0, "y": -5.0, "amplitude": 0.2},
]

x_radar = np.linspace(x_start, x_end, number_of_pulses)
# Collect radar positions at each measurement (virtual element position)
xyz_radar = np.column_stack((
    x_radar,                          # The varying x positions
    np.full(number_of_pulses, y_radar),
    np.full(number_of_pulses, z_radar),
))
# Calculate some distances to limit the range axis
radar_r0 = np.sqrt( y_radar**2 + z_radar**2 )
radar_r1 = np.linalg.norm(xyz_radar[0,:])

# Range/Time window definitions
range0 = 0           # m
range_window = radar_r1 + 50 # m to simulate
total_time = 2.0 * range_window / c  # two-way time over 'range_window'
num_samples = int(sampling_rate * total_time)

# Time and range axes
t = np.linspace(0, total_time, num_samples, endpoint=False)
range_axis = np.linspace(range0, range0 + (c * total_time / 2), num_samples, endpoint=False)

# We'll simulate a sinc shaped pattern in range. 
# This corresponds to range-compressed data with a main-lobe width.
sinc_width = 2.0 * range_resolution / c
def sinc_pulse(time_array, center, width):
    """
    Generates a normalized sinc function centered at 'center' with 'width'.
    """
    y = np.sinc((time_array - center) / width)
    # Normalize maximum amplitude to 1.0 for that single pulse
    peak = np.max(np.abs(y))
    if peak > 0:
        y /= peak
    return y

# Simulate 2D array of pulses vs. time delay (range)
# Use a complex64 dtype to store complex data
range_compressed_data = np.zeros((number_of_pulses, num_samples), dtype=np.complex64)

for i in range(number_of_pulses):
    # We'll sum up echoes from all targets for the i-th pulse in complex form
    echo_sum = np.zeros(num_samples, dtype=np.complex64)
    
    # Radar's position in x at pulse i, y is fixed
    radar_x = x_radar[i]
    
    for tgt in targets:
        # Compute range from the radar to this target
        r = np.sqrt((radar_x - tgt["x"])**2 + (y_radar - tgt["y"])**2 + z_radar**2 )
        # Two-way time delay
        delay = 2.0 * r / c

        # Complex phasor for phase progression (two-way)
        target_phasor = np.exp(1j * 2 * np.pi * (2 * r / wavelength))
        
        # Generate echo from this target, scaled by its amplitude
        echo_sum += tgt["amplitude"] * sinc_pulse(t, delay, sinc_width) * target_phasor
    
    # Assign the summed echo to the pulses_2d array
    range_compressed_data[i, :] = echo_sum

# Plot the 2D pulses vs range
# Use the magnitude for plotting
data_to_plot = np.abs(range_compressed_data)

fig, ax = plt.subplots(figsize=(10, 6))

im = ax.imshow(
    data_to_plot,
    extent=[range_axis[0], range_axis[-1], 0, number_of_pulses],
    aspect='auto',
    origin='lower',
    cmap='jet'
)
ax.set_xlabel("Range (m)")
ax.set_ylabel("Pulse Number")
ax.set_title("Radar Echo Simulation with Multiple Targets (Magnitude)")

# Color bar for amplitude
cbar = plt.colorbar(im, ax=ax)
cbar.set_label("Amplitude (Magnitude)")

plt.tight_layout()
plt.show()


Now it’s time to **focus our data and turn it into a sharp image**! 🎯✨  

Below, you’ll find a function to perform SAR beamforming—our very first **SAR processor**! 🦸‍♂️🦸‍♀️ It follows the exact procedure we outlined above.  

In [ ]:
def sar_beamforming(range_vector, range_compressed_data_array, xyz_radar, wavelength, 
                         image_x_extent, image_y_extent, nb_pixels_x, nb_pixels_y):
    """
    Form a 2D image by time-domain delay and sum beamforming (also called back-projection)
    """
    # Number of pulses and range bins in data
    number_of_pulses = range_compressed_data_array.shape[0]
    nb_range_samples = range_compressed_data_array.shape[1]

    # Range sampling
    range_spacing = range_vector[1] - range_vector[0]  # Assume uniform spacing

    # Create output image
    image = np.zeros((nb_pixels_y, nb_pixels_x), dtype=np.complex128)

    # Define x and y pixel coordinates
    x_grid = np.linspace(image_x_extent[0], image_x_extent[-1], nb_pixels_x)
    y_grid = np.linspace(image_y_extent[0], image_y_extent[-1], nb_pixels_y)

    # Loop over each pixel
    for iy, y_val in enumerate(y_grid):
        for ix, x_val in enumerate(x_grid):
            # Loop through the virtual antenna elements (pulses)
            for pulse in range(number_of_pulses):
                # Range from this radar position to the pixel
                pixel_range = np.sqrt((x_val - xyz_radar[pulse,0])**2 + 
                                      (y_val - xyz_radar[pulse,1])**2 + 
                                      (xyz_radar[pulse,2])**2)
                ## Find corresponding sample index (nearest neighbor)
                sample_index = int(round((pixel_range-range_vector[0])/range_spacing))
                if 0 <= sample_index < nb_range_samples:
                    # Calculate the phase we expected an echo from this pixel to have
                    pixel_phase = 2 * np.pi * (2*pixel_range) / wavelength
                    # Use a phasor with opposite phase to align the echoes from all virtual elements
                    pixel_phasor_correction = np.exp(-1j*pixel_phase)
                    # Sum the phase aligned echoes togehter
                    image[iy, ix] += range_compressed_data_array[pulse, sample_index] * pixel_phasor_correction

    image /= number_of_pulses # Normalize image amplitude

    return image, x_grid, y_grid

Feel free to **experiment** with the settings! 🎮✨  

You can play around with:  
- 📐 The **image extent** (how big of an area we want to image).  
- 🎯 The **target configuration** (where we place the targets).  
- 🔢 The **number of pixels** we use to form the image.  

Watch those echoes **compress into twinkling stars**! 🌟  

And yes, you’ll notice that the target responses in the image have **sidelobes**. Why?  

Because even though we’ve focused our **huge virtual array** into a narrow beam, **every antenna (even our synthetic one)** has sidelobes. 🌊📡 

Every time we **combine a finite number of wave signals**—whether from a **limited number of sources** or a **finite aperture**—we end up with **sidelobe energy**.  

Let’s see SAR beamfroming magic in action! 🪄✨  

In [ ]:
# Define image extent
x_extent = (-15, 15)
y_extent = (-15, 15)
# Define number of pixels. 
# In reality these should be based on resolution, but here you can play around with these
nb_pixels_x = 100
nb_pixels_y = 100

# Perform the back-projection
im, x_grid, y_grid = sar_beamforming(
    range_axis,
    range_compressed_data,
    xyz_radar,
    wavelength,
    x_extent,
    y_extent,
    nb_pixels_x,
    nb_pixels_y
)

# Plot the SAR image
plt.figure(figsize=(8, 6))

# Plot the SAR image
plt.imshow(
    np.abs(im),
    extent=[x_grid[0], x_grid[-1], y_grid[0], y_grid[-1]],
    origin='lower',
    aspect='auto',
    cmap='jet'
)

plt.colorbar(label="Amplitude (Magnitude)")
plt.xlabel("x (m)")
plt.ylabel("y (m)")
plt.title("Back-Projection Image (Magnitude)")
plt.tight_layout()
plt.show()

### Visualizing the beamfroming process 🔍

Below is another illustration of how **SAR beamforming process**. 📡✨  

We’re zooming in on a **small 20x20 pixel patch** from the scene center, where one of the targets is located. 🎯  

In this example we **focus the beam on different pixels one by one** (though, in practice, this can be done in parallel).  

Here's what happens for each pixel:  
1. 🔍 We **trace the time-delay (range) path** in the data, identifying and picking the samples corresponding to echoes from the pixel location.  
2. 🔄 We **apply phase corrections** to each sample along this range path to align them properly.  
3. ➕ We **sum the phase-corrected samples**, producing a single complex value (**phasor**) representing the energy received from that pixel location.  

This process is repeated for **every pixel** to build up the complete SAR image. 🖼️✨  

In [ ]:
import matplotlib.animation as animation
from IPython.display import HTML, display
from matplotlib.patches import FancyArrowPatch
import matplotlib as mpl

# Increase the animation embed limit to, say, 100 MB:
mpl.rcParams['animation.embed_limit'] = 100

# Pick a 20x20 subregion around the center
nb_pixels_y, nb_pixels_x = len(y_grid), len(x_grid)
sub_size = 20

center_y = nb_pixels_y // 2
center_x = nb_pixels_x // 2

start_y = max(center_y - sub_size // 2, 0)
end_y = min(start_y + sub_size, nb_pixels_y)

start_x = max(center_x - sub_size // 2, 0)
end_x = min(start_x + sub_size, nb_pixels_x)

sub_nb_pixels_y = end_y - start_y
sub_nb_pixels_x = end_x - start_x

# Sub-image coordinate arrays
y_subgrid = y_grid[start_y:end_y]
x_subgrid = x_grid[start_x:end_x]

# Create arrays only for the subregion
center_sums = np.zeros((sub_nb_pixels_y, sub_nb_pixels_x), dtype=np.complex128)
center_distances = np.zeros((sub_nb_pixels_y, sub_nb_pixels_x, number_of_pulses), dtype=np.float32)

range_spacing = range_axis[1] - range_axis[0] 

# Compute back-projection for only the 20x20 patch (also store the pixel ranges)
for iy in range(sub_nb_pixels_y):
    y_val = y_subgrid[iy]
    for ix in range(sub_nb_pixels_x):
        x_val = x_subgrid[ix]
        # Sum over pulses (coherent back-projection)
        for p in range(number_of_pulses):
            dist = np.sqrt((x_val - xyz_radar[p,0])**2 + (y_val - xyz_radar[p,1])**2 + xyz_radar[p,2]**2)
            center_distances[iy, ix, p] = dist
            sample_index = int(round((dist - range0) / range_spacing))
            if 0 <= sample_index < num_samples:
                phase_corr = np.exp(-1j * 2 * np.pi * (2 * dist / wavelength))
                center_sums[iy, ix] += range_compressed_data[p, sample_index] * phase_corr

# Now put everything in place for the animation
animation_image = np.zeros_like(center_sums, dtype=np.complex128)
frames_count = sub_nb_pixels_y * sub_nb_pixels_x

fig = plt.figure(figsize=(14, 4))

# Back-Projection image patch
ax1 = plt.subplot(1, 3, 1)
bp_img = ax1.imshow(
    np.abs(animation_image),
    extent=[x_subgrid[0], x_subgrid[-1], y_subgrid[0], y_subgrid[-1]],
    origin='lower',
    aspect='auto',
    cmap='jet'
)
bp_img.set_clim(0, np.max(np.abs(center_sums)))
ax1.set_title("Center Patch (20x20) - In Progress")
ax1.set_xlabel("x (m)")
ax1.set_ylabel("y (m)")
cbar1 = plt.colorbar(bp_img, ax=ax1)
cbar1.set_label("Amplitude")

range_min = 125
range_max = 155

# Range-compressed data with a pixel range trace
ax2 = plt.subplot(1, 3, 2)
raw_im = ax2.imshow(
    data_to_plot,
    extent=[range_axis[0], range_axis[-1], 0, number_of_pulses],
    aspect='auto',
    origin='lower',
    cmap='jet'
)
ax2.set_xlim([range_min, range_max])
ax2.set_title("Raw Data (magnitude)\n+ Pixel Range Trace (white line)")
ax2.set_xlabel("Range (m)")
ax2.set_ylabel("Pulse Number")
cbar2 = plt.colorbar(raw_im, ax=ax2)
cbar2.set_label("Amplitude")
range_trace_line, = ax2.plot([], [], 'w-', linewidth=5)

# Phasor sum
ax3 = plt.subplot(1, 3, 3)
ax3.set_title("Phasor Sum for Current Pixel")
ax3.set_xlabel("Real")
ax3.set_ylabel("Imag")
ax3.set_xlim([-number_of_pulses, number_of_pulses])
ax3.set_ylim([-number_of_pulses, number_of_pulses])

phasor_arrow = FancyArrowPatch(
    posA=(0,0),
    posB=(0,0),
    arrowstyle='->',
    mutation_scale=15,  # controls arrow head size
    color='red',
    linewidth=1.5
)
ax3.add_patch(phasor_arrow)

plt.tight_layout()

def init():
    bp_img.set_data(np.abs(animation_image))
    range_trace_line.set_data([], [])
    # For the arrow, we can set it to be a tiny arrow at the origin initially
    phasor_arrow.set_positions((0,0), (0,0))
    return bp_img, range_trace_line, phasor_arrow

def animate(frame_idx):
    """Reveal one pixel (in the 20x20 patch) per frame, then show an arrow."""
    # Identify sub-pixel row/col
    iy_sub = frame_idx // sub_nb_pixels_x
    ix_sub = frame_idx % sub_nb_pixels_x
    
    # Update partial image
    animation_image[iy_sub, ix_sub] = center_sums[iy_sub, ix_sub]
    bp_img.set_array(np.abs(animation_image))

    # Update the range trace
    dist_array = center_distances[iy_sub, ix_sub, :]
    range_trace_line.set_data(dist_array, np.arange(number_of_pulses))

    # Update the arrow
    p_sum = center_sums[iy_sub, ix_sub]
    ph_real, ph_imag = p_sum.real, p_sum.imag
    phasor_arrow.set_positions((0, 0), (ph_real, ph_imag))

    return bp_img, range_trace_line, phasor_arrow

bp_animation = animation.FuncAnimation(
    fig, animate,
    frames=frames_count,
    init_func=init,
    blit=False,
    interval=100
)

HTML(bp_animation.to_jshtml())                

### 🌟📡 SAR Imaging as Beamforming — The Connection to Phased Arrays 📡🌟  

In **Lesson 3**, we explored **beamforming** using phased arrays, where we **separated echoes based on their angle of arrival**. This separation was achieved because **echoes from different angles** traveled **different path lengths**, resulting in **different phase shifts** at each antenna element.  

As we've just learned, **SAR imaging works on the exact same principle!** 🎯  

### 🔍 How It Works:  
- Echoes from different pixel positions on the ground introduce **different time delays and phase shifts** across our **virtual antenna elements** (the positions of the moving radar at each pulse measurement point). 🌍📡  
- The difference compared to our phased array in Lesson 3 is that the time delays and phase shifts **are not constant** between the virtual elements. 📈
- This is because the virtual array **is very large** and we are not considering a **plane wave**. 📐🔄  

**SAR imaging is like beamforming with a large, virtual phased antenna array.** 🛰️🚀
